# Creating your own visualizations

This tutorial describes how to build custom visualizations (`BaseVisualizations`) in OCE. 

We have a unique way of building visualizations in OCE to make them interactive,
shareable, and customizable.

## The BaseVisualization class

### Basic flow

Fundamentally `BaseVisualization`s are rendered via `render_X()` (the function `render_ipynb()` 
returns an output renderable in Jupyter notebooks and the function `render_data_url()` 
returns a dataURL which can be pasted in any web browser and rendered). 

Each `BaseVisualization` is drawn onto a blank div via a JavaScript template
defined by the file specified by the property `JS_NAME`. The `render_X()` functions
compile the HTML/JavaScript code and populates the visualization tempalte with
the visualization data returned by `get_data()`. `save_html` saves the visulalization
to a standalone HTML file.

### Notes on use

Visualizations often require external JavaScript packages to load this can be
specified in `self.packages` and a dictionary of such available packages is in
`package_urls`.

In [1]:
from olorenchemengine import BaseVisualization, log_arguments

class CustomVis(BaseVisualization):

    package_urls = {...}

    @log_arguments
    def __init__(self, log=True, **kwargs):
        self.packages = []

    def get_data() -> dict:
        """Get data for visualization in JSON-like dictionary.
        """
        return {}

    @property
    def JS_NAME(self) -> str:
        pass

    def render(self, data: dict = None, print_html_js=False) -> str:
        pass

    def save_html(self, path):
        pass
    
    def render_ipynb(self, data: dict = None, print_html_js=False) -> str:
       pass

    def render_data_url(self, data: dict = None, print_html_js=False) -> str:
        pass

### Notes on attributes class

You'll notice two methods in the visualizations, `get_attributes()` and 
`from_attributes`. These two methods are used to systematize the arguments used
for visualizations, especially the aesthetic and `BaseClass` subclasses. These
attributes are optional, but highly reccomended for contributed code.

In [2]:

@staticmethod
def get_attributes() -> list:
    pass

@classmethod
def from_attributes(cls, attributes: dict) -> "BaseVisualization":
    pass

## Creating by example

We will go through an example chemical space plot to show the components of a 
visualization. We'll do two run-throughs. The first will show the basic funtional
components of the Python class. The second will show the integration with JavaScript.

### First run-through

In [4]:
# This is what the visualization looks like

from olorenchemengine import ChemicalSpacePlot, BACEDataset, MorganVecRepresentation
vis = ChemicalSpacePlot(BACEDataset(), MorganVecRepresentation())
vis.render_ipynb()

### Second run-through

First we'll show the data that is passed to the HTML/JavaScript created by 
`get_data()` and then we will show the final combined HTML/JavaScript + data file.

In [10]:
# This is what the visualization data looks like
vis.get_data()

{'title': 'Chemical Space Plot',
 'xaxis_title': 'Component 1',
 'yaxis_title': 'Component 2',
 'hover_size': 0.4,
 'xaxis_type': 'linear',
 'yaxis_type': 'linear',
 'width': 800,
 'height': 600,
 'opacity': 1,
 'X': [43.61760330200195,
  -17.472150802612305,
  41.86582946777344,
  158.97496032714844,
  -24.860200881958008,
  -2.232616424560547,
  -35.85402297973633,
  -67.47518157958984,
  55.304420471191406,
  -13.724129676818848,
  -3.4850451946258545,
  -134.3723602294922,
  -62.69178771972656,
  30.23807716369629,
  -5.901937961578369,
  10.940505981445312,
  -4.623172283172607,
  -43.74485778808594,
  8.369648933410645,
  68.97262573242188,
  -22.280351638793945,
  -30.012405395507812,
  75.25151062011719,
  64.52973937988281,
  -3.1073720455169678,
  -26.3098087310791,
  -29.127517700195312,
  14.045056343078613,
  27.511388778686523,
  62.6290168762207,
  46.29264831542969,
  18.152881622314453,
  49.81310272216797,
  -81.98030853271484,
  41.48120880126953,
  89.24085998535156

This is what the combined vis HTML/JavaScript/Data looks like
Notice the first section has a header, then package imports, then a div
with id 'basevis-entry' which is where the visualization is rendered.
The div has the tag data-visdata which has the JSON-ified data returned by
get_data(). Then the visualization is rendered with Plotly with the compounds
rendered on hover with smilesDrawer.

Note that the bulk of the visualization is performed in the JavaScript file
pointed to by `JS_NAME` and the bulk of the processing is done in Python
and returned by get_data. As a general principle, time-intensive procedures should
be performed in the `__init__()` of the visualization with less time-intensive procedures
like data filtering performed in `get_data()`

After the printed HTML/JavaScript is the dataURL which can be copy pasted without 
the quotation marks to be rendered in any browser.

In [9]:
vis.render_data_url(print_html_js=True)


        <!DOCTYPE html>
        <head>
        <meta charset="utf-8">
        </head>

        <html style="height: 100%;">

        <!-- Load packages -->
        <script src="https://cdn.plot.ly/plotly-2.12.1.min.js"></script><script src="https://unpkg.com/smiles-drawer@1.0.10/dist/smiles-drawer.min.js"></script>
        <body style="background-color:#FFFFFF; height: 100%;">
        <!-- Create a div where the graph will take place -->
        <div id="basevis-entry" data-visdata="{&quot;title&quot;: &quot;Chemical Space Plot&quot;, &quot;xaxis_title&quot;: &quot;Component 1&quot;, &quot;yaxis_title&quot;: &quot;Component 2&quot;, &quot;hover_size&quot;: 0.4, &quot;xaxis_type&quot;: &quot;linear&quot;, &quot;yaxis_type&quot;: &quot;linear&quot;, &quot;width&quot;: 800, &quot;height&quot;: 600, &quot;opacity&quot;: 1, &quot;X&quot;: [43.61760330200195, -17.472150802612305, 41.86582946777344, 158.97496032714844, -24.860200881958008, -2.232616424560547, -35.85402297973633, -67.47518157

'data:text/html,%0A%20%20%20%20%20%20%20%20%3C%21DOCTYPE%20html%3E%0A%20%20%20%20%20%20%20%20%3Chead%3E%0A%20%20%20%20%20%20%20%20%3Cmeta%20charset%3D%22utf-8%22%3E%0A%20%20%20%20%20%20%20%20%3C%2Fhead%3E%0A%0A%20%20%20%20%20%20%20%20%3Chtml%20style%3D%22height%3A%20100%25%3B%22%3E%0A%0A%20%20%20%20%20%20%20%20%3C%21--%20Load%20packages%20--%3E%0A%20%20%20%20%20%20%20%20%3Cscript%20src%3D%22https%3A%2F%2Fcdn.plot.ly%2Fplotly-2.12.1.min.js%22%3E%3C%2Fscript%3E%3Cscript%20src%3D%22https%3A%2F%2Funpkg.com%2Fsmiles-drawer%401.0.10%2Fdist%2Fsmiles-drawer.min.js%22%3E%3C%2Fscript%3E%0A%20%20%20%20%20%20%20%20%3Cbody%20style%3D%22background-color%3A%23FFFFFF%3B%20height%3A%20100%25%3B%22%3E%0A%20%20%20%20%20%20%20%20%3C%21--%20Create%20a%20div%20where%20the%20graph%20will%20take%20place%20--%3E%0A%20%20%20%20%20%20%20%20%3Cdiv%20id%3D%22basevis-entry%22%20data-visdata%3D%22%7B%26quot%3Btitle%26quot%3B%3A%20%26quot%3BChemical%20Space%20Plot%26quot%3B%2C%20%26quot%3Bxaxis_title%26quot%3B%3A%20%